In [2]:
from diffusion import experiments
from diffusion.experiments import InferenceExperiment, InferenceExperimentConfiguration
import logging
import os

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

# set notebook autoload
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'pyDOE3'

In [2]:


config = InferenceExperimentConfiguration()

curr_path = "./"

config.path = os.path.join(curr_path, "inference_data")
config.name = "inference_experiment"
config.overwrite_results = True
experiment = InferenceExperiment(logger, config)
experiment.run()

Generating experiments with factors: ['image_size', 'num_inf_steps'] and levels: [[(512, 512)], [25]]
INFO:__main__:Generating experiments with factors: ['image_size', 'num_inf_steps'] and levels: [[(512, 512)], [25]]
DoE: [[0. 0.]]
INFO:__main__:DoE: [[0. 0.]]
Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  4.24it/s]
Running experiments

INFO:__main__:Running experiments

Running experiment: {'image_size': (512, 512), 'num_inf_steps': 25}
INFO:__main__:Running experiment: {'image_size': (512, 512), 'num_inf_steps': 25}
  0%|          | 0/1 [00:00<?, ?it/s]Running batch with prompt: ['a kitchen', 'space']
INFO:__main__:Running batch with prompt: ['a kitchen', 'space']
100%|██████████| 1/1 [00:39<00:00, 39.40s/it]
Saving results
INFO:__main__:Saving results
Saving results to ./inference_data/repetition_0__image_size_(512, 512)__num_inf_steps_25/results.json
INFO:__main__:Saving results to ./inference_data/repetition_0__image_size_(512, 512)__num_inf_steps_25/results.